In [ ]:
from chessf.parser import FilePGN
from chessf.engine import Stockfish

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
torch.manual_seed(0)

import plotly.express as px
from time import perf_counter

from scipy.optimize import curve_fit

In [ ]:
pgn_2017_02 = "pgn/lichess_db_standard_rated_2017-02.pgn"
file = FilePGN(pgn_2017_02)

In [ ]:
stockfish_path = "stockfish/stockfish.exe"
stockfish = Stockfish(stockfish_path)

## Eval CP

In [ ]:
white_won = []
evals = []

for i in range(100_000):
    print(i, end='\r')
    
    game_moves, info = file.get_and_parse_next_good_game()
    stockfish.start_new_game()
    is_white_won = int(info['Result'] == '1-0')
    
    for game_move in game_moves:
    
        eval_type, eval_int = stockfish.get_eval(depth=5)

        if eval_type == 'cp':
            evals.append(eval_int)
            white_won.append(is_white_won)
        
        stockfish.make_pgn_move(game_move)

In [ ]:
df = pd.DataFrame({'material_diffs': evals, 'white_won': white_won})

In [ ]:
g = df.groupby(
    df['material_diffs'].clip(-1200, 1200) // 50 * 50 + 25
).agg({'white_won': 'mean'}).squeeze()

In [ ]:
px.line(g, template='plotly_white')

In [ ]:
def sigmoid(x, a, b):
    return 1 / (1 + np.exp((-x+b)/a))
    
p0 = (300, 10)

popt_1 = curve_fit(sigmoid, g.index.values, g.values, p0=p0)[0]
popt_1

In [ ]:
pred = sigmoid(g.index.values, *popt_1)

In [ ]:
px.line(x=g.index.values, y=[g, pred], template='plotly_white').update_layout(showlegend=False)

In [ ]:
np.mean(np.square(g.values - pred)) * 1000

In [ ]:
popt_1

## Draws

In [ ]:
n_draws = 0
n_games = 500_000
for _ in range(n_games):
    game = file.get_next_game()
    if '[Result "1/2-1/2"]' in game:
        n_draws += 1

In [ ]:
n_draws / n_games

## Rating diff

In [ ]:
diffs = []
for _ in range(20_000):
    moves, info = file.get_and_parse_next_good_game()
    elo_diff = info['WhiteElo'] - info['BlackElo']
    diffs.append(elo_diff)

In [ ]:
px.histogram(diffs)

In [ ]:
np.quantile(diffs, [0.20, 0.40, 0.60, 0.80])